1. Process excel file to csv after finding the 5 key frames from onset to apex and encode emotions to numbers.

In [28]:
import pandas as pd
import numpy as np
def loading_casme_labels(objective_flag):
    # loading_samm_labels(root_db_path, dB, objective_flag)
#     label_filename = 'SAMM_Micro_FACS_Codes_v2.xlsx'

#     label_path = root_db_path + dB + "/" + label_filename
    label_file = pd.read_excel("CASME2-coding-20190701.xlsx", sheet_name="Sheet1")
    # remove class 6, 7
    if objective_flag:
        # print(objective_flag)
        label_file = label_file.loc[label_file['Estimated Emotion'] != "others"]
    subject = label_file[['Subject']]
    filename = label_file[['Filename']]
    label = label_file[['Estimated Emotion']]
    onset = label_file[['OnsetFrame']]
    apex = label_file[['ApexFrame']]
    offset = label_file[['OffsetFrame']]
    # print(label)
    return subject, filename, label, onset, apex, offset


def loading_casme_table(objective_flag):
    # loading_samm_table(root_db_path, dB, objective_flag):
    subject, filename, label, onset, apex, offset = loading_casme_labels(objective_flag)
    # print("subject:%s filename:%s label:%s objective_classes:%s" %(subject, filename, label, objective_classes))
    subject = subject.to_numpy()
    filename = filename.to_numpy()
    label = label.to_numpy()
    table = np.transpose( np.array( [filename, label] ) )
    return table

subject, filename, label, onset, apex, offset= loading_casme_labels(False)

encodedLabels=pd.DataFrame({
    "disgust": 0,
    "fear":1,
    "happiness":2,
    "others":3,
    "repression":4,
    "sadness":5,
    "surprise":6
}, index=[0])
F=pd.DataFrame({"Subject": ["..."], "Clip": ["..."], "Label": ["..."], "ApexFrame": ["..."]})
for (idx1, onsetf), (idx2, apexf), (idx3, subj), (idx4, fname), (idx5, label) in zip(onset.iterrows(), apex.iterrows(), subject.iterrows(), filename.iterrows(), label.iterrows()):
    if(np.issubdtype(type(apexf.values[0]), np.number) and np.issubdtype(type(onsetf.values[0]), np.number)):
        gap = (np.subtract(apexf.values[0], onsetf.values[0]))/5+1
        for i in range(0, 4):
            frame = int(min((onsetf.values[0]+gap*i), apexf.values[0]))
            # Maybe we need it oneday. [Reconstructing path]
            # path="sub"+str(subj.values[0])+"/"+fname.values[0]+"/"+"reg_img"+str(frame)+".jpg"
            subjPath = "sub"+str(subj.values[0]).zfill(2)
            F = pd.concat([F, pd.DataFrame({"Subject": subjPath, "Clip": fname.values[0], "Label": encodedLabels[label].values[0], "ApexFrame": frame})],ignore_index= True)
            # Checking if a frame is out-of-boundaries of onset and apex. Just incase yknow?
            if(frame < onsetf.values[0] or frame > apexf.values[0]):
                print("Frame out of bound:", frame)
        # Maybe we need it oneday. [Reconstructing path]
        # path="sub"+str(subj.values[0])+"/"+fname.values[0]+"/"+"reg_img"+str(apexf.values[0])+".jpg"
        subjPath = "sub"+str(subj.values[0]).zfill(2)
        F = pd.concat([F, pd.DataFrame({"Subject": subjPath, "Clip": fname.values[0], "Label": encodedLabels[label].values[0], "ApexFrame": apexf.values[0]})], ignore_index = True)
# Remove the placeholder row.
finalDf= F.drop([0])
finalDf.to_csv("casme2_five_frames.csv", index=True)

Applying motion magnification on the dataset and save it to another directory so it could be easily accessible for future usage.

In [24]:
from motion_magnify import motion_magnify
import cv2
from pathlib import Path
Path("CASME2_Motion").mkdir(parents=True, exist_ok=True)
for (idx1, onsetf), (idx3, subj), (idx4, fname), (idx5, offsetf)in zip(onset.iterrows(), subject.iterrows(), filename.iterrows(), offset.iterrows()):
    # Merging all frames of a subject case into a single video called combinedFrames.mp4 nested in original folder structure.
    imgPath = "sub"+str(subj.values[0]).zfill(2)+"/"+fname.values[0]+"/reg_img"+str(onsetf.values[0])+".jpg"
    img = cv2.imread("CASME2_Original/Cropped/"+imgPath)
    if not isinstance(img, type(None)):
        h, w, *_ = img.shape
        fourcc = cv2.VideoWriter_fourcc('H','2','6','4')
        basePath = "sub"+str(subj.values[0]).zfill(2)+"/"+fname.values[0]
        Path("CASME2_Motion/videos/"+basePath).mkdir(parents=True, exist_ok=True)
        videoOutPath = basePath+"/combinedFrames.mp4"
        video = cv2.VideoWriter('CASME2_Motion/videos/' + videoOutPath, fourcc, 20, (w, h))
        for j in range(onsetf.values[0],offsetf.values[0]):
            imgPath = "sub"+str(subj.values[0]).zfill(2)+"/"+fname.values[0]+"/reg_img"+str(j)+".jpg"
            img = cv2.imread("CASME2_Original/Cropped/"+imgPath)
            if not isinstance(img, type(None)):
                resizedImage = cv2.resize(img, (w, h))
                video.write(resizedImage)
        cv2.destroyAllWindows()
        video.release()
        # Motion Magnifying the combinedFrames.mp4 video then save another video in the same folder called MMFrames.mp4
        videoOutPathMotion = "sub"+str(subj.values[0]).zfill(2)+"/"+fname.values[0]+"/MMFrames.mp4"
        motion_magnify(video_path="CASME2_Motion/videos/"+videoOutPath,
                   output_path="CASME2_Motion/videos/"+videoOutPathMotion,
                   motion=True,
                   alpha=20,
                   filter_type='iir',
                   r_low=0.05,
                   r_high=0.4,
                   lambda_c=16,
                   fps=20)

        # Read all frames in MMFrames.mp4 and save it to the original dataset structure.
        MMVideo = cv2.VideoCapture("CASME2_Motion/videos/"+videoOutPathMotion)
        success,image = MMVideo.read()
        count = onsetf.values[0]
        while success:
            basePath = "sub"+str(subj.values[0]).zfill(2)+"/"+fname.values[0]
            Path("CASME2_Motion/MMCropped/"+basePath).mkdir(parents=True, exist_ok=True)
            imgPath =  basePath+"/reg_img"+str(count)+".jpg"
            cv2.imwrite("CASME2_Motion/MMCropped/"+imgPath, image)     # save frame as JPEG file
            success,image = MMVideo.read()
            count += 1


In [62]:
from sklearn.model_selection import train_test_split
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

df=pd.read_csv("casme2_five_frames.csv")
train_set,test_set=train_test_split(df,test_size=0.25)
img_folder='CASME2_Original/Cropped/'
BATCH_SIZE=32

class CASME2Dataset(Dataset):
  def __init__(self,csv,img_folder,transform):
    self.csv=csv
    self.transform=transform
    self.img_folder=img_folder
    self.image_names=self.csv[:][['Subject', 'Clip', 'ApexFrame']]
    self.labels=np.array(self.csv.drop(['Id','Subject', 'Clip', 'ApexFrame'], axis=1))

  def __len__(self):
    return len(self.image_names)

  def __getitem__(self,index):
    idx=self.image_names.iloc[index]
    constructedpath = str(idx['Subject'])+"/"+idx['Clip']+"/"+"reg_img"+str(idx['ApexFrame'])
    dsimage=cv2.imread(self.img_folder+constructedpath+'.jpg')
    # dsimage=cv2.cvtColor(dsimage,cv2.COLOR_BGR2RGB)

    dsimage=self.transform(dsimage)
    targets=self.labels[index]

    sample = {'image': dsimage,'labels':targets}

    return sample

train_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((400, 400)),
                transforms.ToTensor()])
test_transform =transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((400, 400)),
                transforms.ToTensor()])
train_dataset = CASME2Dataset(csv=train_set,img_folder=img_folder, transform=train_transform)
test_dataset = CASME2Dataset(csv=test_set,img_folder=img_folder, transform=test_transform)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=True
)
print(train_dataloader)
for i_batch, sample_batched in enumerate(train_dataloader):
    print(i_batch, sample_batched)

0 {'image': tensor([[[[0.3490, 0.3529, 0.3569,  ..., 0.1882, 0.1725, 0.1608],
          [0.3490, 0.3529, 0.3529,  ..., 0.1961, 0.1804, 0.1686],
          [0.3490, 0.3529, 0.3529,  ..., 0.2078, 0.1922, 0.1804],
          ...,
          [0.0157, 0.0196, 0.0196,  ..., 0.0471, 0.0471, 0.0510],
          [0.0157, 0.0157, 0.0157,  ..., 0.0471, 0.0471, 0.0510],
          [0.0157, 0.0157, 0.0157,  ..., 0.0510, 0.0510, 0.0549]],

         [[0.4275, 0.4314, 0.4392,  ..., 0.2235, 0.2078, 0.1961],
          [0.4275, 0.4314, 0.4353,  ..., 0.2314, 0.2157, 0.2039],
          [0.4275, 0.4314, 0.4353,  ..., 0.2431, 0.2275, 0.2157],
          ...,
          [0.0196, 0.0235, 0.0235,  ..., 0.0549, 0.0549, 0.0588],
          [0.0196, 0.0196, 0.0196,  ..., 0.0549, 0.0549, 0.0588],
          [0.0196, 0.0196, 0.0196,  ..., 0.0588, 0.0588, 0.0627]],

         [[0.5647, 0.5686, 0.5765,  ..., 0.2784, 0.2588, 0.2471],
          [0.5647, 0.5686, 0.5725,  ..., 0.2863, 0.2667, 0.2549],
          [0.5647, 0.5686, 0.5

TypeError: pic should be Tensor or ndarray. Got <class 'NoneType'>.